In [ ]:
from datetime import datetime, timedelta,date
import pandas as pd
import jenkspy
%matplotlib inline
from sklearn.metrics import classification_report,confusion_matrix
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.cluster import KMeans
import chart_studio.plotly as py
import plotly.offline as pyoff
import plotly.graph_objs as go

In [ ]:
import xgboost as xgb
from sklearn.model_selection import KFold, cross_val_score, train_test_split

In [ ]:
pyoff.init_notebook_mode()

In [ ]:
tx_data = pd.read_csv('Part4.csv')

In [ ]:
tx_data

In [ ]:
tx_data = tx_data.rename(columns={"帳戶名稱": "CustomerID", "結帳日期": "InvoiceDate","總價": "Revenue"})

In [ ]:
tx_data['InvoiceDate'] = pd.to_datetime(tx_data['InvoiceDate'])

In [ ]:
tx_data['InvoiceDate'].describe()

In [ ]:
tx_uk = tx_data

In [ ]:
tx_3m = tx_uk[(tx_uk.InvoiceDate < pd.to_datetime('2019-1-1')) & (tx_uk.InvoiceDate >= pd.to_datetime('2017-1-1'))].reset_index(drop=True)
tx_6m = tx_uk[(tx_uk.InvoiceDate >= pd.to_datetime('2019-1-1')) & (tx_uk.InvoiceDate < pd.to_datetime('2020-10-1'))].reset_index(drop=True)

In [ ]:
tx_3m['InvoiceDate'].describe()

In [ ]:
tx_user = pd.DataFrame(tx_3m['CustomerID'].unique())
tx_user.columns = ['CustomerID']

# Recency

In [ ]:
tx_max_purchase = tx_3m.groupby('CustomerID').InvoiceDate.max().reset_index()

In [ ]:
tx_max_purchase.columns = ['CustomerID','MaxPurchaseDate']

In [ ]:
tx_max_purchase['Recency'] = (tx_max_purchase['MaxPurchaseDate'].max() - tx_max_purchase['MaxPurchaseDate']).dt.days

In [ ]:
tx_user = pd.merge(tx_user, tx_max_purchase[['CustomerID','Recency']], on='CustomerID')

In [ ]:
tx_user.head()

In [ ]:
tx_user.Recency.describe()

In [ ]:
plot_data = [
    go.Histogram(
        x=tx_user['Recency']
    )
]

plot_layout = go.Layout(
        title='Recency'
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
pyoff.iplot(fig)

In [ ]:
def goodness_of_variance_fit(array, classes):
    # get the break points
    classes = jenkspy.jenks_breaks(array, classes)

    # do the actual classification
    classified = np.array([classify(i, classes) for i in array])

    # max value of zones
    maxz = max(classified)

    # nested list of zone indices
    zone_indices = [[idx for idx, val in enumerate(classified) if zone + 1 == val] for zone in range(maxz)]

    # sum of squared deviations from array mean
    sdam = np.sum((array - array.mean()) ** 2)

    # sorted polygon stats
    array_sort = [np.array([array[index] for index in zone]) for zone in zone_indices]

    # sum of squared deviations of class means
    sdcm = sum([np.sum((classified - classified.mean()) ** 2) for classified in array_sort])

    # goodness of variance fit
    gvf = (sdam - sdcm) / sdam

    return gvf

def classify(value, breaks):
    for i in range(1, len(breaks)):
        if value < breaks[i]:
            return i
    return len(breaks) - 1

In [ ]:
array_org = np.array(tx_user[['Recency']].T)[0]
list_GVF = [] 
gvf = 0.0
nclasses = 2
while gvf < .99:
    gvf = goodness_of_variance_fit(array_org, nclasses)
    print(nclasses,gvf)
    list_GVF = list_GVF + [gvf]
    nclasses += 1
plt.figure()
plt.xlim(2,len(list_GVF)+1)
plt.plot([x+2 for x in range(len(list_GVF))], list_GVF)
plt.plot([x+2 for x in range(len(list_GVF))], [0.95]*len(list_GVF))
plt.xlabel("Number of cluster")
plt.show()

In [ ]:
bins = jenkspy.jenks_breaks(array_org, 6)
labels = [x for x in range(len(bins)-1)]
tx_user['RecencyCluster'] = pd.cut(tx_user['Recency'], bins=bins, labels=labels)

In [ ]:
tx_user.groupby('RecencyCluster')['Recency'].describe()

In [ ]:
def order_cluster(cluster_field_name, target_field_name,df,ascending):
    new_cluster_field_name = 'new_' + cluster_field_name
    df_new = df.groupby(cluster_field_name)[target_field_name].mean().reset_index()
    df_new = df_new.sort_values(by=target_field_name,ascending=ascending).reset_index(drop=True)
    df_new['index'] = df_new.index
    df_final = pd.merge(df,df_new[[cluster_field_name,'index']], on=cluster_field_name)
    df_final = df_final.drop([cluster_field_name],axis=1)
    df_final = df_final.rename(columns={"index":cluster_field_name})
    return df_final


In [ ]:
tx_user = order_cluster('RecencyCluster', 'Recency',tx_user,False)

# Frequency

In [ ]:
tx_frequency = tx_3m.groupby('CustomerID').InvoiceDate.count().reset_index()

In [ ]:
tx_frequency.columns = ['CustomerID','Frequency']

In [ ]:
tx_frequency.head()

In [ ]:
tx_user = pd.merge(tx_user, tx_frequency, on='CustomerID')

In [ ]:
tx_user.head()

In [ ]:
tx_user.Frequency.describe()

In [ ]:
plot_data = [
    go.Histogram(
        x=tx_user.query('Frequency < 1000')['Frequency']
    )
]

plot_layout = go.Layout(
        title='Frequency'
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
pyoff.iplot(fig)

In [ ]:
array_org = np.array(tx_user[['Frequency']].T)[0]
list_GVF = [] 
gvf = 0.0
nclasses = 2
while gvf < .99:
    gvf = goodness_of_variance_fit(array_org, nclasses)
    print(nclasses,gvf)
    list_GVF = list_GVF + [gvf]
    nclasses += 1
plt.figure()
plt.xlim(2,len(list_GVF)+1)
plt.plot([x+2 for x in range(len(list_GVF))], list_GVF)
plt.plot([x+2 for x in range(len(list_GVF))], [0.95]*len(list_GVF))
plt.xlabel("Number of cluster")
plt.show()

In [ ]:
bins = jenkspy.jenks_breaks(array_org, 6)
labels = [x for x in range(len(bins)-1)]
tx_user['FrequencyCluster'] = pd.cut(tx_user['Frequency'], bins=bins, labels=labels)

In [ ]:
tx_user.groupby('FrequencyCluster')['Frequency'].describe()

In [ ]:
tx_user = order_cluster('FrequencyCluster', 'Frequency',tx_user,True)

# Monetary Value

In [ ]:
tx_revenue = tx_3m.groupby('CustomerID').Revenue.sum().reset_index()

In [ ]:
tx_revenue.head()

In [ ]:
tx_user = pd.merge(tx_user, tx_revenue, on='CustomerID')

In [ ]:
tx_user.Revenue.describe()

In [ ]:
plot_data = [
    go.Histogram(
        x=tx_user.query('Revenue < 10000')['Revenue']
    )
]

plot_layout = go.Layout(
        title='Monetary Value'
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
pyoff.iplot(fig)

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
array_org = np.array(tx_user[['Revenue']].T)[0]
list_GVF = [] 
gvf = 0.0
nclasses = 2
while gvf < .95:
    gvf = goodness_of_variance_fit(array_org, nclasses)
    print(nclasses,gvf)
    list_GVF = list_GVF + [gvf]
    nclasses += 1
plt.figure()
plt.xlim(2,len(list_GVF))
plt.plot([x+2 for x in range(len(list_GVF))], list_GVF)
plt.plot([x+2 for x in range(len(list_GVF))], [0.8]*len(list_GVF))
plt.xlabel("Number of cluster")
plt.show()

In [ ]:
bins = jenkspy.jenks_breaks(array_org, 6)
labels = [x for x in range(len(bins)-1)]
tx_user['RevenueCluster'] = pd.cut(tx_user['Revenue'], bins=bins, labels=labels)

In [ ]:
tx_user = order_cluster('RevenueCluster', 'Revenue',tx_user,True)

In [ ]:
tx_user.groupby('RevenueCluster')['Revenue'].describe()

# Overall Segmentation

In [ ]:
tx_user.head()

In [ ]:
tx_user['OverallScore'] = tx_user['RecencyCluster'] + tx_user['FrequencyCluster'] + tx_user['RevenueCluster']

In [ ]:
tx_user.groupby('OverallScore')['Recency','Frequency','Revenue'].mean()

In [ ]:
tx_user.groupby('OverallScore')['Recency'].count()

In [ ]:
tx_user['Segment'] = 'Low-Value'
tx_user.loc[tx_user['OverallScore']>3,'Segment'] = 'Mid-Value' 
tx_user.loc[tx_user['OverallScore']>7,'Segment'] = 'High-Value' 

In [ ]:
tx_graph = tx_user.query("Revenue < 50000 and Frequency < 2000")

plot_data = [
    go.Scatter(
        x=tx_graph.query("Segment == 'Low-Value'")['Frequency'],
        y=tx_graph.query("Segment == 'Low-Value'")['Revenue'],
        mode='markers',
        name='Low',
        marker= dict(size= 7,
            line= dict(width=1),
            color= 'blue',
            opacity= 0.8
           )
    ),
        go.Scatter(
        x=tx_graph.query("Segment == 'Mid-Value'")['Frequency'],
        y=tx_graph.query("Segment == 'Mid-Value'")['Revenue'],
        mode='markers',
        name='Mid',
        marker= dict(size= 9,
            line= dict(width=1),
            color= 'green',
            opacity= 0.5
           )
    ),
        go.Scatter(
        x=tx_graph.query("Segment == 'High-Value'")['Frequency'],
        y=tx_graph.query("Segment == 'High-Value'")['Revenue'],
        mode='markers',
        name='High',
        marker= dict(size= 11,
            line= dict(width=1),
            color= 'red',
            opacity= 0.9
           )
    ),
]

plot_layout = go.Layout(
        yaxis= {'title': "Revenue"},
        xaxis= {'title': "Frequency"},
        title='Segments'
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
pyoff.iplot(fig)

In [ ]:
tx_graph = tx_user.query("Revenue < 50000 and Frequency < 2000")

plot_data = [
    go.Scatter(
        x=tx_graph.query("Segment == 'Low-Value'")['Recency'],
        y=tx_graph.query("Segment == 'Low-Value'")['Revenue'],
        mode='markers',
        name='Low',
        marker= dict(size= 7,
            line= dict(width=1),
            color= 'blue',
            opacity= 0.8
           )
    ),
        go.Scatter(
        x=tx_graph.query("Segment == 'Mid-Value'")['Recency'],
        y=tx_graph.query("Segment == 'Mid-Value'")['Revenue'],
        mode='markers',
        name='Mid',
        marker= dict(size= 9,
            line= dict(width=1),
            color= 'green',
            opacity= 0.5
           )
    ),
        go.Scatter(
        x=tx_graph.query("Segment == 'High-Value'")['Recency'],
        y=tx_graph.query("Segment == 'High-Value'")['Revenue'],
        mode='markers',
        name='High',
        marker= dict(size= 11,
            line= dict(width=1),
            color= 'red',
            opacity= 0.9
           )
    ),
]

plot_layout = go.Layout(
        yaxis= {'title': "Revenue"},
        xaxis= {'title': "Recency"},
        title='Segments'
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
pyoff.iplot(fig)

In [ ]:
tx_graph = tx_user.query("Revenue < 50000 and Frequency < 2000")

plot_data = [
    go.Scatter(
        x=tx_graph.query("Segment == 'Low-Value'")['Recency'],
        y=tx_graph.query("Segment == 'Low-Value'")['Frequency'],
        mode='markers',
        name='Low',
        marker= dict(size= 7,
            line= dict(width=1),
            color= 'blue',
            opacity= 0.8
           )
    ),
        go.Scatter(
        x=tx_graph.query("Segment == 'Mid-Value'")['Recency'],
        y=tx_graph.query("Segment == 'Mid-Value'")['Frequency'],
        mode='markers',
        name='Mid',
        marker= dict(size= 9,
            line= dict(width=1),
            color= 'green',
            opacity= 0.5
           )
    ),
        go.Scatter(
        x=tx_graph.query("Segment == 'High-Value'")['Recency'],
        y=tx_graph.query("Segment == 'High-Value'")['Frequency'],
        mode='markers',
        name='High',
        marker= dict(size= 11,
            line= dict(width=1),
            color= 'red',
            opacity= 0.9
           )
    ),
]

plot_layout = go.Layout(
        yaxis= {'title': "Frequency"},
        xaxis= {'title': "Recency"},
        title='Segments'
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
pyoff.iplot(fig)

# LTV

In [ ]:
tx_user.head()

In [ ]:
tx_user_6m = tx_6m.groupby('CustomerID')['Revenue'].sum().reset_index()
tx_user_6m.columns = ['CustomerID','m6_Revenue']

In [ ]:
tx_user_6m.head()

In [ ]:
plot_data = [
    go.Histogram(
        x=tx_user_6m.query('m6_Revenue < 10000')['m6_Revenue']
    )
]

plot_layout = go.Layout(
        title='6m Revenue'
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
pyoff.iplot(fig)

In [ ]:
tx_merge = pd.merge(tx_user, tx_user_6m, on='CustomerID', how='left')

In [ ]:
tx_merge.head()

In [ ]:
tx_merge = tx_merge.fillna(0)

In [ ]:
tx_merge.groupby('Segment')['m6_Revenue'].mean()

In [ ]:
tx_graph = tx_merge.query("m6_Revenue < 30000")

plot_data = [
    go.Scatter(
        x=tx_graph.query("Segment == 'Low-Value'")['OverallScore'],
        y=tx_graph.query("Segment == 'Low-Value'")['m6_Revenue'],
        mode='markers',
        name='Low',
        marker= dict(size= 7,
            line= dict(width=1),
            color= 'blue',
            opacity= 0.8
           )
    ),
        go.Scatter(
        x=tx_graph.query("Segment == 'Mid-Value'")['OverallScore'],
        y=tx_graph.query("Segment == 'Mid-Value'")['m6_Revenue'],
        mode='markers',
        name='Mid',
        marker= dict(size= 9,
            line= dict(width=1),
            color= 'green',
            opacity= 0.5
           )
    ),
        go.Scatter(
        x=tx_graph.query("Segment == 'High-Value'")['OverallScore'],
        y=tx_graph.query("Segment == 'High-Value'")['m6_Revenue'],
        mode='markers',
        name='High',
        marker= dict(size= 11,
            line= dict(width=1),
            color= 'red',
            opacity= 0.9
           )
    ),
]

plot_layout = go.Layout(
        yaxis= {'title': "6m LTV"},
        xaxis= {'title': "RFM Score"},
        title='LTV'
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
pyoff.iplot(fig)

In [ ]:
tx_merge = tx_merge[tx_merge['m6_Revenue']<tx_merge['m6_Revenue'].quantile(0.99)]

In [ ]:
kmeans = KMeans(n_clusters=3)
kmeans.fit(tx_merge[['m6_Revenue']])
tx_merge['LTVCluster'] = kmeans.predict(tx_merge[['m6_Revenue']])

In [ ]:
tx_merge = order_cluster('LTVCluster', 'm6_Revenue',tx_merge,True)

In [ ]:
tx_merge.groupby('LTVCluster')['m6_Revenue'].describe()

In [ ]:
tx_cluster = tx_merge.copy()

In [ ]:
tx_cluster.head()

In [ ]:
tx_cluster.groupby('LTVCluster')['m6_Revenue'].describe()

In [ ]:
tx_cluster.head()

In [ ]:
list(tx_cluster.columns[:])

In [ ]:
tx_class = pd.get_dummies(tx_cluster[list(tx_cluster.columns[1:])])
temp_df = pd.DataFrame()
temp_df["CustomerID"] = tx_cluster["CustomerID"].astype('category').cat.codes
tx_class = pd.concat([temp_df,tx_class],axis=1)

In [ ]:
tx_class.head()

In [ ]:
corr_matrix = tx_class.corr()
corr_matrix['LTVCluster'].sort_values(ascending=False)

In [ ]:
X = tx_class.drop(['LTVCluster','m6_Revenue'],axis=1)
y = tx_class['LTVCluster']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=56)

In [ ]:
ltv_xgb_model = xgb.XGBClassifier(max_depth=5, learning_rate=0.05,objective= 'multi:softprob',n_jobs=-1).fit(X_train, y_train)

print('Accuracy of XGB classifier on training set: {:.2f}'
       .format(ltv_xgb_model.score(X_train, y_train)))
print('Accuracy of XGB classifier on test set: {:.2f}'
       .format(ltv_xgb_model.score(X_test[X_train.columns], y_test)))



In [ ]:
tx_class.groupby(['LTVCluster'])['CustomerID'].count()/tx_class.CustomerID.count()

In [ ]:
y_pred = ltv_xgb_model.predict(X_test)

In [ ]:
print (classification_report(y_test, y_pred))

# hyperparameter

In [ ]:
import time

import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.25, random_state=1)
# x_train, y_train, x_valid, y_valid, x_test, y_test =  train_test_split(X, y, test_size=0.05, random_state=56)

clf = xgb.XGBClassifier()

param_grid = {
        'silent': [False],
        'max_depth': [6, 10, 15, 20],
        'learning_rate': [0.001, 0.01, 0.1, 0.2, 0,3],
        'subsample': [0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
        'colsample_bytree': [0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
        'colsample_bylevel': [0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
        'min_child_weight': [0.5, 1.0, 3.0, 5.0, 7.0, 10.0],
        'gamma': [0, 0.25, 0.5, 1.0],
        'reg_lambda': [0.1, 1.0, 5.0, 10.0, 50.0, 100.0],
        'n_estimators': [100]}

fit_params = {'eval_metric': 'mlogloss',
              'early_stopping_rounds': 10,
              'eval_set': [(x_valid, y_valid)]}

rs_clf = RandomizedSearchCV(clf, param_grid, n_iter=20,
                            n_jobs=1, verbose=2, cv=2,
                            fit_params=fit_params,
                            scoring='neg_log_loss', refit=False, random_state=42)
print("Randomized search..")
search_time_start = time.time()
rs_clf.fit(x_train, y_train)
print("Randomized search time:", time.time() - search_time_start)

best_score = rs_clf.best_score_
best_params = rs_clf.best_params_
print("Best score: {}".format(best_score))
print("Best params: ")
for param_name in sorted(best_params.keys()):
    print('%s: %r' % (param_name, best_params[param_name]))

In [ ]:
ltv_xgb_model = xgb.XGBClassifier(colsample_bylevel= 1.0, colsample_bytree= 0.5, gamma= 0.25, learning_rate= 0.1
 , max_depth= 15, min_child_weight= 7.0, n_estimators= 100 , reg_lambda= 10.0, silent= False 
 ,subsample= 1.0, objective= 'multi:softprob', n_jobs=-1 ).fit(x_train, y_train)

print('Accuracy of XGB classifier on training set: {:.2f}'
       .format(ltv_xgb_model.score(x_train, y_train)))
print('Accuracy of XGB classifier on test set: {:.2f}'
       .format(ltv_xgb_model.score(x_test[X_train.columns], y_test)))

